In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
# Create the first table
all_time_prices_query = '''
CREATE TABLE all_time_prices (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock_Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(all_time_prices_query)

conn.commit()


In [2]:
#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
all_time_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits', 'Ticker']
all_time = pd.DataFrame(columns = all_time_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker
        info.rename({'Stock Splits':'Stock_Splits'},axis = 1,inplace = True)
        print(info) 
        
        # Append the first dataframe to the table
        all_time = pd.concat([all_time, info])

        #print(momentum_append)
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    except AttributeError: 
        print("Attribute error found")
        continue
    stop = time.time()
    duration = stop-start
    print(duration)
    break

#PUSH TO DATABASE
all_time.to_sql('all_time_prices', conn, if_exists='append', index=False)
# Close the connection
conn.close()

AOS: No data found for this date range, symbol may be delisted
Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, Ticker]
Index: []
0.1748189926147461


In [3]:
all_time.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          0 non-null      object 
 1   Open          0 non-null      object 
 2   High          0 non-null      object 
 3   Low           0 non-null      object 
 4   Close         0 non-null      object 
 5   Volume        0 non-null      object 
 6   Dividends     0 non-null      object 
 7   Stock_Splits  0 non-null      object 
 8   Ticker        0 non-null      object 
 9   Adj Close     0 non-null      float64
dtypes: float64(1), object(9)
memory usage: 108.0+ bytes


In [4]:
all_time

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Ticker,Adj Close
